In [ ]:
# Imports
#  Note: these are all the imports you need! Do not import any other functions / packages
%matplotlib inline

import numpy as np
import pandas as pd
import geopy as gp
from geopy.distance import great_circle


In [ ]:
zillowpd = pd.read_csv('../raw_data/Neighborhood_MedianValuePerSqft_AllHomes.csv') 


In [ ]:
zillowpdsd = zillowpd[zillowpd['CountyName'] == 'San Diego'] # remove all counties not san diego
# remove uneccessary columns
zillowpdsd = zillowpdsd.drop('RegionID', 1)
zillowpdsd = zillowpdsd.drop('State', 1)
zillowpdsd = zillowpdsd.drop('SizeRank', 1)
# remove all but latest rent info
zillowpdsd2 = zillowpdsd[['RegionName', 'City','Metro','CountyName','2017-03']]
zillowpdsd2

In [ ]:
zillowpdsd.to_csv('../clean_data/clean_full_pricepersqft.csv')
zillowpdsd2.to_csv('../clean_data/clean_latest_pricepersqft.csv')

In [42]:
randomPoints1 = pd.read_csv('../clean_data/randomPoints.csv') 
open_pot = pd.read_csv('../clean_data/cleaned_get_it_done_open_potholes.csv')


In [43]:
randResults = np.zeros((len(randomPoints1),len(open_pot)))

In [44]:
for i in randomPoints1.index:
    randomPoint = randomPoints1.iloc[i]
    for j in open_pot.index:
        open_pott = open_pot.iloc[j]
        randResults[i][j] = great_circle((randomPoint['Y'], randomPoint['X']), (open_pott['lat'], open_pott['long'])).meters
np.save('../clean_data/randDistances.npy',busResults)


In [45]:
randResults

array([[ 21294.18137354,  32455.46854985,  19346.96391187, ...,
         11223.40486153,   4243.650653  ,             nan],
       [ 26758.55166862,  17773.62264483,  30353.9422605 , ...,
         38644.98415844,  44216.64858176,             nan],
       [ 19296.32118276,  11328.90529847,  23086.43584364, ...,
         31269.88935603,  36747.12363651,             nan],
       ..., 
       [ 14985.92497456,  25946.68759569,  14482.13546138, ...,
          5142.64485801,   4415.40279766,             nan],
       [ 15316.04657951,  17580.02277632,  22240.26965402, ...,
         12270.31128578,  21779.5897994 ,             nan],
       [ 12680.86413256,  13145.31330652,  20398.13083886, ...,
         13633.39893837,  23011.1173002 ,             nan]])